# **Força Elástica**

Este projeto realiza a análise da deformação elástica de uma mola helicoidal em função da força aplicada.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
rigidez = pd.read_csv('/content/drive/MyDrive/Cursos_Udemy/Redes_Neurais/mola.csv', sep=',', encoding='iso-8859-1')

## **Exploração e tratamento dos Dados**

In [ ]:
type(rigidez)

In [ ]:
rigidez.head(26)

In [ ]:
rigidez.shape

In [ ]:
rigidez2=rigidez.drop('cargas', axis=1)

In [ ]:
rigidez2=rigidez2.drop('Lo',axis=1)

In [ ]:
rigidez2.head(26)

In [ ]:
rigidez2.dtypes


## **Análises estatísticas**

In [ ]:
from scipy import stats

In [ ]:
rigidez2.describe()

In [ ]:
rigidez2.K.hist(bins=8)
plt.title('Rigidez da mola')
plt.show()

In [ ]:
import plotly.express as px
fig =  px.histogram (rigidez2 ,  x = "K", nbins=12) 
fig.update_layout(width=400,height=400)
fig.show()

In [ ]:
sns.distplot(rigidez2['K']);

In [ ]:
stats.probplot(rigidez2['K'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot")
plt.show()

In [ ]:
fig = px.box(rigidez2, y='K')
fig.update_layout(width=400,height=400)
fig.show()

## **Correlações**

In [ ]:
plt.scatter(rigidez2.x,rigidez2.forca)
plt.xlabel('Deformação(mm)')
plt.ylabel('Força(N)')
plt.title('Rigidez da mola')
plt.grid(True)
plt.show()


In [ ]:
coef_correlacao = rigidez2['x'].corr(rigidez2['forca'], method = 'kendall')
print(coef_correlacao)

Parâmetros corr:
method = pearson, spearmann e kendall. Default: pearson.
- Pearson: dados paramétricos (exemplo: distribuição normal)
- Spearman: grande quantidade de dados não paramétricos.
- Kendall: pequena quantidade de dados não paramétricos.

In [ ]:
coef_determinacao = coef_correlacao**2
coef_determinacao

In [ ]:
sns.pairplot(rigidez2);

In [ ]:
correlacoes = rigidez2.corr(method = 'kendall')

In [ ]:
correlacoes

In [ ]:
plt.figure()
sns.heatmap(correlacoes, annot=True);

## **Redes Neurais Artificiais: Regressão**

In [ ]:
rigidez2.head()

In [ ]:
x = rigidez2.iloc[:, 2:3].values
y = rigidez2.iloc[:, 0:1].values

In [ ]:
x

In [ ]:
x.shape

In [ ]:
y

In [ ]:
y.shape

In [ ]:
from sklearn.neural_network import MLPRegressor


In [ ]:
regressor = MLPRegressor(hidden_layer_sizes=(26), activation='identity', verbose=True,
                         batch_size=1, solver='lbfgs', random_state = 12)


In [ ]:
regressor.fit(x, y)

In [ ]:
regressor.n_layers_

In [ ]:
regressor.score(x, y)

In [ ]:
deformacao = 0.06

In [ ]:
previsao_forca = regressor.predict(np.array(deformacao).reshape(1,1))

In [ ]:
previsao_forca

In [ ]:

plt.scatter(x, y)
plt.plot(x, regressor.predict(x), color = 'red')
plt.title('Rigidez')
plt.xlabel('Deformação (mm)')
plt.ylabel('Força (N)');

In [ ]:
regressor.fit(y,x)

In [ ]:
forca = 0.1

In [ ]:
previsao_deformacao = regressor.predict(np.array(forca).reshape(1,1))
previsao_deformacao

## **Padronizando a escala**

In [ ]:
rigidez2.head(26)

In [ ]:
x = rigidez2.iloc[:, 2:3].values
y = rigidez2.iloc[:, 0:1].values

In [ ]:
from sklearn.preprocessing import StandardScaler

O StandardScaler transforma seus dados de forma que sua distribuição tenha um valor médio 0 e desvio padrão de 1. Aproxima para uma distribuição normal.

In [ ]:
scaler_x = StandardScaler()
x = scaler_x.fit_transform(x)

In [ ]:
x

In [ ]:
scaler_y = StandardScaler()
y = scaler_y.fit_transform(y)

In [ ]:
y

In [ ]:
from sklearn.neural_network import MLPRegressor


In [ ]:
regressor = MLPRegressor(hidden_layer_sizes=(26), activation='identity', verbose=True,
                         batch_size=1, solver='lbfgs', random_state = 12)


In [ ]:
regressor.fit(x, y)

In [ ]:
regressor.score(x, y)

In [ ]:
deformacao = 0.005

In [ ]:
previsao_forca = scaler_y.inverse_transform(regressor.predict(scaler_x.transform(np.array(deformacao).reshape(1, 1))))

In [ ]:
previsao_forca

In [ ]:
plt.scatter(x, y)
plt.plot(x, regressor.predict(x), color = 'red')
plt.title('Rigidez')
plt.xlabel('Deformação (mm)')
plt.ylabel('Força (N)');